In [46]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# MLflow Regression Pipeline Notebook

This notebook runs the MLflow Regression Pipeline on Databricks and inspects its results. For more information about the MLflow Regression Pipeline, including usage examples, see the [Regression Pipeline overview documentation](https://mlflow.org/docs/latest/pipelines.html#regression-pipeline) the [Regression Pipeline API documentation](https://mlflow.org/docs/latest/python_api/mlflow.pipelines.html#module-mlflow.pipelines.regression.v1.pipeline).

In [47]:
!C:\Users\GEBETSBE\AppData\Local\Programs\Python\Python39\Scripts\mlflow --version

mlflow, version 1.28.0


In [48]:
from mlflow.pipelines import Pipeline
import os

chocoportable_path = "C:\\ProgramData\\chocoportable\\bin\\"
if chocoportable_path not in os.environ['PATH']:
    os.environ['PATH'] += ";" + chocoportable_path

p = Pipeline(profile="local")

2022/08/16 10:44:51 INFO mlflow.pipelines.pipeline: Creating MLflow Pipeline 'mlp-regression-template' with profile: 'local'


In [49]:
p.clean()

In [50]:
p.inspect()

In [51]:
p.run("ingest")

2022/08/16 10:44:57 INFO mlflow.pipelines.steps.ingest.datasets: Resolving input data from '['C:\\Users\\GEBETSBE\\Documents\\Dev\\mlp-regression-template\\data\\combined_disasters.csv']'
2022/08/16 10:44:57 INFO mlflow.pipelines.steps.ingest.datasets: Resolved input data to 'C:\Users\GEBETSBE\AppData\Local\Temp\tmpjc9aty54\combined_disasters.csv'
2022/08/16 10:44:57 INFO mlflow.pipelines.steps.ingest.datasets: Converting dataset to parquet format, if necessary
Loading dataset CSV using `pandas.read_csv()` with default arguments and assumed index column 0 which may not produce the desired schema. If the schema is not correct, you can adjust it by modifying the `load_file_as_dataframe()` function in `steps/ingest.py`
2022/08/16 10:45:04 INFO mlflow.pipelines.steps.ingest: Successfully stored data in parquet format at 'C:\Users\GEBETSBE\.mlflow\pipelines\202617ce1eed864f31ad8970a0482c9fc9fca989f030c04989a44ab6a7a1c30d\steps\ingest\outputs\dataset.parquet'
2022/08/16 10:45:04 INFO mlflow.

In [52]:
p.run("split")

2022/08/16 10:45:15 INFO mlflow.pipelines.steps.split: Creating hash buckets on input dataset containing 94332 rows consumes 0.3820223808288574 seconds.


In [53]:
p.run("transform")

In [54]:
p.run("train")

2022/08/16 10:45:32 INFO mlflow.store.db.utils: Creating initial MLflow database tables...
2022/08/16 10:45:32 INFO mlflow.store.db.utils: Updating database tables
INFO  [alembic.runtime.migration] Context impl SQLiteImpl.
INFO  [alembic.runtime.migration] Will assume non-transactional DDL.
INFO  [alembic.runtime.migration] Running upgrade  -> 451aebb31d03, add metric step
INFO  [alembic.runtime.migration] Running upgrade 451aebb31d03 -> 90e64c465722, migrate user column to tags
INFO  [alembic.runtime.migration] Running upgrade 90e64c465722 -> 181f10493468, allow nulls for metric values
INFO  [alembic.runtime.migration] Running upgrade 181f10493468 -> df50e92ffc5e, Add Experiment Tags Table
INFO  [alembic.runtime.migration] Running upgrade df50e92ffc5e -> 7ac759974ad8, Update run tags with larger limit
INFO  [alembic.runtime.migration] Running upgrade 7ac759974ad8 -> 89d4b8295536, create latest metrics table
INFO  [89d4b8295536_create_latest_metrics_table_py] Migration complete!
INFO  

Metric,training,validation
root_mean_squared_error,0.132428,0.132823
example_count,75697,9459
max_error,0.867156,0.868787
mean_absolute_error,0.101394,0.102031
mean_absolute_percentage_error,1.51077e+12,2.40634e+12
mean_on_label,0.852313,0.852654
mean_squared_error,0.0175372,0.0176421
r2_score,0.015397,0.0186385
score,0.015397,0.0186385
sum_on_label,64517.5,8065.25


In [55]:
p.run("evaluate")

2022/08/16 10:45:59 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.
2022/08/16 10:46:00 INFO mlflow.models.evaluation.default_evaluator: Shap explainer _PatchedKernelExplainer is used.

100%|##########| 10/10 [00:00<00:00, 332.89it/s]
Unable to serialize underlying model using MLflow, will use SHAP serialization
Setuptools is replacing distutils.
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
2022/08/16 10:46:05 INFO mlflow.models.evaluation.base: Evaluating the model with the default evaluator.


Metric,validation,test
root_mean_squared_error,0.132823,0.135512
example_count,9459,9176.000000
max_error,0.868787,0.863262
mean_absolute_error,0.102031,0.102741
mean_absolute_percentage_error,2.40634e+12,2890454671776.986816
mean_on_label,0.852654,0.851678
mean_squared_error,0.0176421,0.018363
r2_score,0.0186385,0.015772
score,0.0186385,0.015772
sum_on_label,8065.25,7815.000000


In [56]:
p.run("register")

Successfully registered model 'taxi_fare_regressor'.
2022/08/16 10:46:09 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: taxi_fare_regressor, version 1
Created version '1' of model 'taxi_fare_regressor'.


In [57]:
p.inspect("train")

Metric,training,validation
root_mean_squared_error,0.132428,0.132823
example_count,75697,9459
max_error,0.867156,0.868787
mean_absolute_error,0.101394,0.102031
mean_absolute_percentage_error,1.51077e+12,2.40634e+12
mean_on_label,0.852313,0.852654
mean_squared_error,0.0175372,0.0176421
r2_score,0.015397,0.0186385
score,0.015397,0.0186385
sum_on_label,64517.5,8065.25


In [58]:
test_data = p.get_artifact("test_data")
test_data.describe()

,roughYear,continent,disasterType
count,9176.000000,9176.000000,9176.000000
mean,0.851678,4.160963,12.319420
std,0.136601,1.463349,6.854619
min,0.000000,0.000000,0.000000
25%,0.833333,3.000000,9.000000
50%,0.833333,5.000000,12.000000
75%,1.000000,5.000000,19.000000
max,1.000000,6.000000,27.000000


In [59]:
trained_model = p.get_artifact("model")
print(trained_model)

mlflow.pyfunc.loaded_model:
  artifact_path: train/model
  flavor: mlflow.sklearn
  run_id: 9ebabc4aadfb432baee67edc838bc97a

